# データオーギュメンテーション

適用するデータオーギュメンテーションの数だけ，以下のコードを修正して実行する．

今回はBlur, RandomBrightness, RandomContrastの３パターンを適用して，元画像を合わせて訓練データを４倍にする．

### 変更が必要な箇所
- save_dir
- track_path
- transform
- cv2.imwrite(save_aug_dir + "/" + os.path.splitext(os.path.basename(image))[0] + "_" + str(RandomBrightness) + ".jpg", a2)のstr()の部分
  

In [ ]:
import matplotlib.pyplot as plt
import albumentations as A
import numpy as np
import cv2
import glob
import os
from natsort import natsorted

# 保存先の用意
save_dir = './aug_brightness_Flip_Flip/'
os.makedirs(save_dir, exist_ok=True)

# ファイル名用の対応表
blur = 0
RandomBrightness = 1
saturation = 2
RandomContrast = 3
HorizontalFlip = 4
Rotate = 5
RandomBrightnessContrast = 6
Contrast_Flip = 7
Flip_Rotate = 8
Bright_Blur = 9
Blur_Rotate = 10
Cont_Blur_Rot = 11
Sat_Bright_Flip = 12
Random = 13

# track内リスト
track_path = "C:/Users/sugie/PycharmProjects/StrongSORT-YOLO/data/exp1/"
track_list = os.listdir(track_path)
track_dir = [f for f in track_list if os.path.isdir(os.path.join(track_path, f))]

# person内のフォルダリスト取得
for exp in track_dir:
    person_path = track_path + exp + "/crops/person/"
    filelist = []
    for d in os.listdir(person_path):
        if os.path.isdir(os.path.join(person_path, d)):
            filelist.append(d)
    person_list = os.listdir(person_path)  # person内のフォルダリスト
    person_dir = [f for f in natsorted(person_list) if os.path.isdir(os.path.join(person_path, f))]

    # ID内の画像の全パスを取得
    for name in person_dir:
        name_path = person_path + name
        name_list = os.listdir(name_path)  # name内のリスト
        name_dir = [f for f in natsorted(name_list) if os.path.isdir(os.path.join(name_path, f))]
        firstloop = True
        file_name_col = []
        point_ID = []
        for ID in name_dir:
            # 保存先の用意
            save_aug_dir = save_dir + exp + "/crops/person/" + name + "/" + ID
            os.makedirs(save_aug_dir, exist_ok=True)
            ID_path = glob.glob(name_path + "/" + ID + "/**")  # ID_path=画像の絶対パスのリスト
            print(ID)
            for image in ID_path:
                img = cv2.imread(image, cv2.IMREAD_COLOR)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.uint8)

                sat_param = (-10, 10)
                rot_param = (1, 10)
                transform = A.Compose([
#ここで適用を設定できます。
#見た目変化
#０：ブラー:blur（default = 3,7）→ぼかし用のガウスカーネルの最大サイズ
#１：明るさ(輝度)、RandomBrightness  (default = -0.2,02)
#２：彩度、HueSaturationValue,sat_shift_limit  (-10,10)
#３：コントラスト、RandomCotrast（default = -0.2, 0.2）

                   # A.HueSaturationValue(sat_shift_limit = sat_param, p=1),
                    A.RandomBrightness(p=1),
                   # A.HorizontalFlip(p=1),
                     #A.Blur(p=1),
                    #A.RandomContrast(p=1),
                    # A.Rotate(limit=rot_param, p=1),
                ])

                a = transform(image=img)
                a1 = a["image"]
                a2 = cv2.cvtColor(a1, cv2.COLOR_RGB2BGR)
                #適用した処理によってstr()内の変数を変更，上記の対応表を見て
                cv2.imwrite(save_aug_dir + "/" + os.path.splitext(os.path.basename(image))[0] + "_" + str(RandomBrightness) + ".jpg", a2)